In [1]:
import time
import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()

In [6]:
wikimatrix = pd.read_csv('/content/WikiMatrix.en-ru.tsv', sep='\t', on_bad_lines='skip')

In [7]:
wikimatrix = wikimatrix.set_axis(['score', 'en', 'ru'], axis=1)

In [8]:
wikimatrix.head()

,score,en,ru
0,1.213647,Fear of the Lord is aking to wonder (or awe).,"Воистину, мучений от твоего Господа надлежит о..."
1,1.197960,"I think I washed his body 50 times.""","Я думаю, что я омыла его тело 50 раз.»"
2,1.195492,There has come to you clear evidence from your...,К вам пришло ясное знамение от вашего Господа.
3,1.194159,"15,000 attend dawn service.","15,000 attend dawn service (англ.)."
4,1.191620,And in the mountains they suffer a calamity.,И в горах их постигла беда.


In [9]:
len(wikimatrix)

5079186

In [3]:
russian_pairs = pd.read_csv('russian_pairs.csv').drop(columns=['Unnamed: 0'])
english_pairs = pd.read_csv('english_pairs.csv').drop(columns=['Unnamed: 0'])

In [4]:
russian_terms = russian_pairs['term'].tolist()
english_terms = english_pairs['term'].tolist()

In [5]:
def check_if_terms_in_sent(sent, ru_terms, en_terms):
    ru_terms_in_sent = any(True for term in ru_terms if term in sent.lower())
    if ru_terms_in_sent:
        en_terms_in_sent = any(True for term in en_terms if term in sent.lower())
        return en_terms_in_sent
    return False

In [10]:
wikimatrix['term_presence'] = wikimatrix.progress_apply(
    lambda x: check_if_terms_in_sent(
        str(x['en']) + ' ' + str(x['ru']),
        russian_terms,
        english_terms
        ),
    axis=1)

  0%|          | 0/5079186 [00:00<?, ?it/s]

In [11]:
wikimatrix.to_csv('wikimatrix_boolean.csv', sep='\t')

In [12]:
wikimatrix_filtered = wikimatrix.loc[wikimatrix.term_presence]

In [14]:
wikimatrix_filtered.to_csv('wikimatrix_filtered.csv', sep='\t')

In [16]:
wikimatrix_filtered

,score,en,ru,term_presence
41,1.172456,"Indeed, I see you and your people to be in man...","Я вижу, что ты и твой народ находитесь в очеви...",True
81,1.167228,No longer can my ears hear the Green Word.,Мои уши больше не могут слышать Зелёное слово.,True
96,1.165858,Or should the system wait a bit and try to del...,Или система должна немного подождать и попытат...,True
111,1.164944,This is the only story of mine whose moral I k...,«How Deep Is Your Love?»,True
168,1.160693,Tell us in not more than 250 words why your gi...,"Расскажите нам не более чем в 250 словах, поче...",True
...,...,...,...,...
5079167,1.020000,"Previously, Veneman was the United States Secr...",Член Американского философского общества (с 20...,True
5079168,1.020000,The explosives plant produced dynamite for som...,Именно эти машины в определенной степени обесп...,True
5079176,1.020000,"Accordingly, the Core emphasizes the developme...","В основном, под культурой понимают человеческу...",True
5079178,1.020000,If a taxpayer changes their definition of qual...,Если аудитор считает необходимым модифицироват...,True
